In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import Layout, Box, Label, Text, Button, VBox, Output

# Define Siamese Network architecture
class SiameseNetwork(nn.Module):
    def __init__(self, input_dim):
        super(SiameseNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x1, x2):
        x = torch.abs(x1 - x2)
        x = self.fc(x)
        return torch.sigmoid(x)

# Load dataset
def load_dataset(file_path):
    dataset = pd.read_csv(file_path)
    return dataset

# Preprocess dataset
def preprocess_dataset(dataset):
    dataset['instruction'] = dataset['instruction'].str.lower()
    dataset['response'] = dataset['response'].str.lower()
    dataset = dataset.drop_duplicates().reset_index(drop=True)
    return dataset

# Vectorize responses using TF-IDF
def vectorize_responses(dataset):
    vectorizer = TfidfVectorizer()
    response_vectors = vectorizer.fit_transform(dataset['response'])
    return vectorizer, response_vectors

# Train Siamese Network
def train_siamese_network(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for data in train_loader:
            inputs1, inputs2, labels = data
            optimizer.zero_grad()
            outputs = model(inputs1, inputs2)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss}")

# Evaluate Siamese Network
def evaluate_siamese_network(model, val_loader):
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for data in val_loader:
            inputs1, inputs2, labels = data
            outputs = model(inputs1, inputs2)
            predicted = torch.round(outputs.squeeze())
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
        accuracy = total_correct / total_samples
        print(f"Validation Accuracy: {accuracy}")

# Retrieve response using Siamese Network
def retrieve_response(user_input, dataset, model, vectorizer):
    user_input_vector = vectorizer.transform([user_input]).toarray()
    response_vectors = vectorizer.transform(dataset['response']).toarray()
    similarities = []
    for response_vector in response_vectors:
        similarity = cosine_similarity(user_input_vector, response_vector.reshape(1, -1))
        similarities.append(similarity[0][0])
    max_similarity_index = np.argmax(similarities)
    response = dataset.iloc[max_similarity_index]['response']
    return response

# Main function
def main():
    # Load and preprocess dataset
    dataset = load_dataset("dataset2.csv")
    dataset = preprocess_dataset(dataset)

    # Vectorize responses using TF-IDF
    vectorizer, response_vectors = vectorize_responses(dataset)

    # Split dataset into train and validation sets
    train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

    # Define Siamese Network
    input_dim = response_vectors.shape[1]
    model = SiameseNetwork(input_dim)

    # Convert dataset to PyTorch tensors
    train_inputs1 = torch.tensor(vectorizer.transform(train_data['instruction']).toarray(), dtype=torch.float32)
    train_inputs2 = torch.tensor(response_vectors[train_data.index].toarray(), dtype=torch.float32)
    train_labels = torch.tensor(train_data.index.values == train_data.index.values, dtype=torch.float32)
    val_inputs1 = torch.tensor(vectorizer.transform(val_data['instruction']).toarray(), dtype=torch.float32)
    val_inputs2 = torch.tensor(response_vectors[val_data.index].toarray(), dtype=torch.float32)
    val_labels = torch.tensor(val_data.index.values == val_data.index.values, dtype=torch.float32)

    # Define DataLoader for training and validation sets
    train_dataset = torch.utils.data.TensorDataset(train_inputs1, train_inputs2, train_labels)
    val_dataset = torch.utils.data.TensorDataset(val_inputs1, val_inputs2, val_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)

    # Define loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train Siamese Network
    train_siamese_network(model, train_loader, criterion, optimizer)

    # Evaluate Siamese Network
    evaluate_siamese_network(model, val_loader)

    # Create widgets
    user_input_text = Text(description="You:", layout=Layout(flex='5 1 auto', width='auto'))
    send_button = Button(description="Send", layout=Layout(flex='1 1 auto', width='auto'))
    exit_button = Button(description="Exit", layout=Layout(flex='1 1 auto', width='auto'))
    output = Output()

    # Define callback functions
    def send_message(sender):
        user_input = user_input_text.value.strip()
        if user_input.lower() == "exit":
            with output:
                print("\033[91m\033[1mChatbot: Goodbye!\033[0m")
                print("")
                print("")
            return
        with output:
            response = retrieve_response(user_input, dataset, model, vectorizer)
            print("\033[92m\033[1mChatbot:", response, "\033[0m")
            print("")
            print("")
            user_input_text.value = ''

    def exit_chat(sender):
        with output:
            print("\033[91m\033[1mChatbot: Goodbye!\033[0m")
            print("")
            print("")

    # Bind callbacks to buttons
    send_button.on_click(send_message)
    exit_button.on_click(exit_chat)

    # Arrange widgets in a box layout
    input_box = Box([Label(value="Your message:"), user_input_text, send_button, exit_button], layout=Layout(display='flex', flex_flow='row', align_items='center'))
    screen_layout = VBox([input_box, output])

    # Display screen layout
    display(screen_layout)

if __name__ == "__main__":
    main()


Epoch 1, Loss: 17.948616936802864
Epoch 2, Loss: 1.8402599627152085
Epoch 3, Loss: 0.0843526131939143
Epoch 4, Loss: 0.034897226258181036
Epoch 5, Loss: 0.019814656872767955
Epoch 6, Loss: 0.012416176090482622
Epoch 7, Loss: 0.008387810623389669
Epoch 8, Loss: 0.005973867155262269
Epoch 9, Loss: 0.004455323156435043
Epoch 10, Loss: 0.0034076034207828343
Validation Accuracy: 1.0
